In [2]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('data/airline_delay_train - airline_delay_train_new.csv',parse_dates =['FlightDate'])

In [124]:
df.head()

,FlightDate,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,Day_of_Week
0,2010-01-17,17:05,MQ,CVG,DFW,812,1,Sunday
1,2010-01-29,17:03,MQ,OMA,ORD,416,0,Friday
2,2010-01-31,18:03,US,SJC,PHX,622,0,Sunday
3,2010-01-26,16:42,YV,MTJ,DEN,197,0,Tuesday
4,2010-01-06,17:53,US,PHL,ORD,678,0,Wednesday


In [15]:
df['dep_delayed_15min'].isna().sum()

0

In [16]:
#Setting target
# remove null targets 
# OTV 
# Pipeline 
# optimization

In [138]:
y = df['dep_delayed_15min']
y = y.to_frame()

In [116]:
#X = df.drop(['dep_delayed_15min'], axis=1)

In [142]:
X = df

In [86]:
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits = 3,test_size= 3 )

In [88]:
categorical_features = df.select_dtypes(include=['object']).columns
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns

In [89]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [147]:
from sklearn.ensemble import RandomForestClassifier

baseline_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier())])

In [148]:
score = []
for fold, (train_index, test_index) in enumerate(tscv.split(X)):
    print("Fold: {}".format(fold))
    print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
    print("\n")
    X_train, X_test = X.iloc[train_index] ,X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    baseline_pipeline.fit(X_train, y_train)
    predict = baseline_pipeline.predict_proba(X_test)
    baseline_score = log_loss(y_test, predict, labels=[0,1])
    print("Fold: {}".format(fold), "Score :{}".format(baseline_score))
    score.append(baseline_score)

/opt/anaconda3/envs/fsds/lib/python3.8/site-packages/sklearn/pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  Index(['Distance', 'dep_delayed_15min'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['DepTime', 'UniqueCarrier', 'Origin', 'Dest', 'Day_of_Week'], dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [149]:
all_backtest = np.mean(score)
all_backtest

In [112]:
baseline_score = log_loss(y_test, predict, labels=[0,1])
baseline_score

0.003350111951167816